<b>

<p>
<center>
<font size="5">
Intro to Data Minning (DATS 6103), Spring 2020
</font>
</center>
</p>

<p>
<center>
<font size="4">
Project Baltimore
</font>
</center>
</p>

<p>
<center>
<font size="3">
Present by:
Pauline Mckim ,
Malikat Coulibaly ,
Spencer Stucky ,
junhe zhang
</font>
</center>
</p>

<p>
<center>
<font size="3">
Due: April 22th
</font>
</center>
</p>

</b>

# Overview
- write me:
- write me:
- **write me**
- write me

# Notebook Configuration

## Google Drive

In [242]:
# skip this if know usin google colab
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Implement me
# Set working directory
abspath = '/content/drive/My Drive/Colab Notebooks/dats6103_project/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Warning

In [0]:
import warnings

# Ignore warnings
warnings.filterwarnings('ignore')

## Matplotlib

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline 

# Set matplotlib sizes
plt.rc('font', size=20)
plt.rc('axes', titlesize=20)
plt.rc('axes', labelsize=20)
plt.rc('xtick', labelsize=20)
plt.rc('ytick', labelsize=20)
plt.rc('legend', fontsize=20)
plt.rc('figure', titlesize=20)

# Data Preprocessing

## Loading data

In [245]:
import pandas as pd

# Load the cleaned raw data
df_balti = pd.read_csv(abspath + 'data/balti_clean_weather.csv', header=0)
df_balti = df_balti.copy(deep=True)
print(df_balti.shape)
df_balti.head()

(100765, 24)


,Arrest,Age,Sex,Race,ArrestDate,ArrestTime,ArrestLocation,IncidentLocation,Charge,District,Post,Neighborhood,Longitude,Latitude,Location 1,Season,Daynight,Max.TemperatureF,Mean.TemperatureF,Min.TemperatureF,PrecipitationIn,Unnamed: 5,Unnamed: 6,ChargeDescriptionClean
0,17174890.0,21.0,F,W,11/4/17,23:50,NaN,NaN,10077,NaN,NaN,NaN,NaN,NaN,NaN,Fall,Night,56,52.2,47,0.00,NaN,NaN,Failure to appear
1,17174874.0,26.0,M,B,11/4/17,23:39,5700 HAZELWOOD CIR,5700 HAZELWOOD CIR,11415,Northeastern,444.0,Frankford,-76.531782,39.339139,"(39.3391387384981, -76.5317819993018)",Fall,Night,56,52.2,47,0.00,NaN,NaN,assualt 2nd degree 1
2,17174889.0,23.0,F,B,11/4/17,23:30,NaN,NaN,10077,NaN,NaN,NaN,NaN,NaN,NaN,Fall,Night,56,52.2,47,0.00,NaN,NaN,Failure to appear
3,17174864.0,33.0,M,B,11/4/17,23:15,NaN,2700 ASHLAND AVE,10077,NaN,NaN,NaN,NaN,NaN,NaN,Fall,Night,56,52.2,47,0.00,NaN,NaN,Failure to appear
4,17174860.0,33.0,M,B,11/4/17,23:15,NaN,5700 EASTERN AVE,10077,NaN,NaN,NaN,NaN,NaN,NaN,Fall,Night,56,52.2,47,0.00,NaN,NaN,Failure to appear


## Getting the name of the target

In [246]:
# info and set target
df_balti.info()
target = 'Charge'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100765 entries, 0 to 100764
Data columns (total 24 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Arrest                  100762 non-null  float64
 1   Age                     100752 non-null  float64
 2   Sex                     100765 non-null  object 
 3   Race                    100765 non-null  object 
 4   ArrestDate              100765 non-null  object 
 5   ArrestTime              100765 non-null  object 
 6   ArrestLocation          60194 non-null   object 
 7   IncidentLocation        63902 non-null   object 
 8   Charge                  100765 non-null  object 
 9   District                60184 non-null   object 
 10  Post                    60171 non-null   float64
 11  Neighborhood            60182 non-null   object 
 12  Longitude               60234 non-null   float64
 13  Latitude                60234 non-null   float64
 14  Location 1          

## Dividing the balti data into training and testing

In [247]:

from sklearn.model_selection import train_test_split

# set seed to 42, so that the result is reproduceable
seed = 42
# Divide the balti data into training (90%) and testing (10%)
df_train, df_test = train_test_split(df_balti, train_size=0.9, random_state=seed, stratify=df_balti[target])

# Reset the index
df_train, df_test = df_train.reset_index(drop=True), df_test.reset_index(drop=True)

print(df_train.shape)
print(df_test.shape)
print(df_balti.shape)
df_train.head()
df_test.head()

(90688, 24)
(10077, 24)
(100765, 24)


,Arrest,Age,Sex,Race,ArrestDate,ArrestTime,ArrestLocation,IncidentLocation,Charge,District,Post,Neighborhood,Longitude,Latitude,Location 1,Season,Daynight,Max.TemperatureF,Mean.TemperatureF,Min.TemperatureF,PrecipitationIn,Unnamed: 5,Unnamed: 6,ChargeDescriptionClean
0,13718446.0,20.0,F,B,7/20/13,15:00,100 E JEFFREY ST,100 E Jeffrey St,30233,Southern,912.0,Brooklyn,-76.611012,39.235427,"(39.2354266707000, -76.6110121835000)",Summer,Day,93,84,74,0,NaN,NaN,Cds:P W/I Dist:Narc
1,13671273.0,53.0,M,B,4/23/13,9:05,3000 W BELVEDERE AVE,3000 W Belvedere Av,43550,Northwestern,633.0,Pimlico Good Neighbors,-76.672573,39.350832,"(39.3508322866000, -76.6725732075000)",Spring,Day,62,54,46,0,NaN,NaN,possess not marihuana
2,14017441.0,42.0,F,B,2/4/14,6:30,NaN,NaN,10077,NaN,NaN,NaN,NaN,NaN,NaN,Winter,Night,35,30,25,0.03,NaN,NaN,Failure to appear
3,14022470.0,26.0,M,W,2/11/14,22:00,1400 BLOOMFIELD AVE,1400 Bloomfield Av,30233,Southwestern,832.0,VIOLETVILLE,-76.666634,39.261553,"(39.2615531016000, -76.6666336321000)",Winter,Night,30,22,14,0,NaN,NaN,Cds:P W/I Dist:Narc
4,17010587.0,26.0,F,B,1/21/17,23:15,2000 POPLAR GROVE ST,2000 POPLAR GROVE ST,11415,Southwestern,811.0,WALBROOK,-76.665885,39.310311,"(39.3103108521708, -76.6658847488879)",Winter,Night,48,46.0,44,0.02,NaN,NaN,assualt 2nd degree 1


## Dividing the training data into training and validating

In [248]:
# set seed to 42, so that the result is reproduceable
seed = 42
# Divide the df_train data into training (80%) and testing (20%)
df_train, df_valid = train_test_split(df_train, train_size=0.8, random_state=seed, stratify=df_train[target])

# Reset the index
df_train, df_valid = df_train.reset_index(drop=True), df_valid.reset_index(drop=True)
df_train.head()
df_valid.head()

,Arrest,Age,Sex,Race,ArrestDate,ArrestTime,ArrestLocation,IncidentLocation,Charge,District,Post,Neighborhood,Longitude,Latitude,Location 1,Season,Daynight,Max.TemperatureF,Mean.TemperatureF,Min.TemperatureF,PrecipitationIn,Unnamed: 5,Unnamed: 6,ChargeDescriptionClean
0,13703767.0,27.0,M,B,6/22/13,13:00,2600 E MONUMENT ST,2600 E Monument St,11635,Eastern,334.0,Madison-Eastend,-76.580779,39.298934,"(39.2989340467000, -76.5807790560000)",Summer,Day,86,74,61,0,NaN,NaN,poss Marijuana
1,17154602.0,52.0,M,B,9/30/17,22:00,2300 MCELDERRY ST,2300 MCELDERRY ST,43550,Southeastern,221.0,McElderry Park,-76.584947,39.297654,"(39.2976544332212, -76.5849470377654)",Fall,Night,65,59.6,54,0.00,NaN,NaN,possess not marihuana
2,14065171.0,18.0,M,B,4/23/14,23:40,NaN,NaN,10077,NaN,NaN,NaN,NaN,NaN,NaN,Spring,Night,60,51,41,0,NaN,NaN,Failure to appear
3,14097983.0,21.0,F,B,6/16/14,10:00,1400 E NORTH AVE,1400 E North Av,20050,Eastern,312.0,EAST BALTIMORE MIDWAY,-76.599223,39.311962,"(39.3119617603000, -76.5992228788000)",Spring,Day,90,75,59,0,NaN,NaN,Dis.Erly Conduct
4,14170148.0,26.0,F,B,10/7/14,16:30,6800 HARFORD RD,6800 Harford Rd,11093,Northeastern,424.0,NORTH HARFORD ROAD,-76.551041,39.364399,"(39.3643990337000, -76.5510410364000)",Fall,Day,69,64,58,0.54,NaN,NaN,General Prostitution


In [249]:
print(df_train.shape)
print(df_valid.shape)
print(df_test.shape)
print(df_balti.shape)

(72550, 24)
(18138, 24)
(10077, 24)
(100765, 24)


## Handling the identifiers

In [0]:
# find identifers in df
def id_checker(df):
    """
    The identifier checker

    Parameters
    ----------
    df : dataframe
    
    Returns
    ----------
    The dataframe of identifiers
    """
    
    # Get the identifiers
    df_id = df[[var for var in df.columns 
                if df[var].nunique(dropna=True) == df[var].notnull().sum()]]
                
    return df_id

## drop identifers

In [251]:
# combine train test
df = pd.concat([df_train, df_valid, df_test], sort=False)

import numpy as np

df_id = id_checker(df)

df_id.head()

# remove id Arrest
df_train = df_train.drop(columns=np.intersect1d(df_train.columns, 'Arrest'))

df_test = df_test.drop(columns=np.intersect1d(df_test.columns, 'Arrest'))

df_valid = df_valid.drop(columns=np.intersect1d(df_valid.columns, 'Arrest'))

df = pd.concat([df_train, df_valid, df_test], sort=False)

print(df.columns)
print(df.shape)

Index(['Age', 'Sex', 'Race', 'ArrestDate', 'ArrestTime', 'ArrestLocation',
       'IncidentLocation', 'Charge', 'District', 'Post', 'Neighborhood',
       'Longitude', 'Latitude', 'Location 1', 'Season', 'Daynight',
       'Max.TemperatureF', 'Mean.TemperatureF', 'Min.TemperatureF',
       'PrecipitationIn', 'Unnamed: 5', 'Unnamed: 6',
       'ChargeDescriptionClean'],
      dtype='object')
(100765, 23)


## Handling date time variables

In [0]:
# handle datetime columns
def datetime_transformer(df, datetime_vars):
    """
    The datetime transformer

    Parameters
    ----------
    df : dataframe
    datetime_vars : the datetime variables
    
    Returns
    ----------
    The dataframe where datetime_vars are transformed into the following 6 datetime types:
    year, month, day, hour, minute and second
    """
    
    # The dictionary with key as datetime type and value as datetime type operator
    dict_ = {'year'   : lambda x : x.dt.year,
             'month'  : lambda x : x.dt.month,
             'day'    : lambda x : x.dt.day,
             'hour'   : lambda x : x.dt.hour,
             'minute' : lambda x : x.dt.minute,
             'second' : lambda x : x.dt.second}
    
    # Make a copy of df
    df_datetime = df.copy(deep=True)
    
    # For each variable in datetime_vars
    for var in datetime_vars:
        # Cast the variable to datetime
        df_datetime[var] = pd.to_datetime(df_datetime[var])
        
        # For each item (datetime_type and datetime_type_operator) in dict_
        for datetime_type, datetime_type_operator in dict_.items():
            # Add a new variable to df_datetime where:
            # the variable's name is var + '_' + datetime_type
            # the variable's values are the ones obtained by datetime_type_operator
            df_datetime[var + '_' + datetime_type] = datetime_type_operator(df_datetime[var])
            
    # Remove datetime_vars from df_datetime
    df_datetime = df_datetime.drop(columns=datetime_vars)
                
    return df_datetime




## encode datetime variables

In [253]:
# import datetime
from datetime import datetime

# clean time columns
def cleanTime(df):
  time = 'ArrestTime'
  if '.' in df[time]:
    return df[time].replace('.',':')
  if '.' not in df[time] and ':' not in df[time]:
    df[time] = df[time] + ':00'
  return df[time]

df['ArrestTime'] = df.apply(cleanTime, axis=1)
df['time'] = df['ArrestDate'] + " " +  df['ArrestTime']
df['datetime'] = df['time'].apply(lambda x: datetime.strptime(x, '%m/%d/%y %H:%M'))

df = datetime_transformer(df, ['datetime'])

# drop encode date and time columns and duplicated location 1
# drop ChargeDescriptionClean because it has 100% possitive correlation with charge which is our target
df = df.drop(columns=['ArrestDate','ArrestTime','time','datetime_second','Location 1','ChargeDescriptionClean', 'Unnamed: 5', 'Unnamed: 6'])

# Separating the training data
df_train = df.iloc[:df_train.shape[0], :].copy(deep=True)

# Separating the validation data
df_valid = df.iloc[df_train.shape[0]:df_train.shape[0] + df_valid.shape[0], :].copy(deep=True)

# Separating the testing data
df_test = df.iloc[df_train.shape[0] + df_valid.shape[0]:, :].copy(deep=True)

df_test.head()

df.head()

,Age,Sex,Race,ArrestLocation,IncidentLocation,Charge,District,Post,Neighborhood,Longitude,Latitude,Season,Daynight,Max.TemperatureF,Mean.TemperatureF,Min.TemperatureF,PrecipitationIn,datetime_year,datetime_month,datetime_day,datetime_hour,datetime_minute
0,34.0,M,B,NaN,NaN,11415,NaN,NaN,NaN,NaN,NaN,Fall,Day,60,51,42,0,2014,10,30,11,0
1,22.0,M,B,1900 W NORTH AVE,W North Av & N Fulton Av,43550,Western,732.0,MONDAWMIN,-76.648292,39.309859,Spring,Night,51,40.9,33,0.00,2016,3,21,23,8
2,37.0,M,B,NaN,NaN,11420,NaN,NaN,NaN,NaN,NaN,Summer,Day,75,67,58,0.03,2013,9,16,9,0
3,26.0,M,B,NaN,NaN,20696,NaN,NaN,NaN,NaN,NaN,Winter,Day,45,34.7,26,0.00,2017,2,14,17,30
4,52.0,M,W,1500 CHARLOTTE AVE,1500 Charlotte Av,43550,Southeastern,234.0,BROENING MANOR,-76.540519,39.276046,Winter,Day,49,36,22,0,2013,12,27,11,0


## Handling missing data
### define nan_checker helper function

In [0]:
# Implement me

def nan_checker(df):
    """
    The NaN checker

    Parameters
    ----------
    df : dataframe
    
    Returns
    ----------
    The dataframe of variables with NaN, their proportion of NaN and dtype
    """
    
    # Get the variables with NaN, their proportion of NaN and dtype
    df_nan = pd.DataFrame([[var, df[var].isna().sum() / df.shape[0], df[var].dtype]
                           for var in df.columns if df[var].isna().sum() > 0],
                          columns=['var', 'proportion', 'dtype'])
    
    # Sort df_nan in accending order of the proportion of NaN
    df_nan = df_nan.sort_values(by='proportion', ascending=False)
    
    return df_nan


# do not remove target variable

### clean missing variable

In [255]:
# Combine df_train, df_valid and df_test
df = pd.concat([df_train, df_valid, df_test], sort=False)
df.head()

# Call nan_checker on df
df_nan = nan_checker(df)

# Print df_nan
df_nan.reset_index(drop=True)

# Get the variables with missing values, their proportion of missing values and dtype
df_miss = df_nan[df_nan['dtype'] == 'float64']

# Print df_miss
print(df_miss.reset_index(drop=True).head())
print(df_nan)


         var  proportion    dtype
0       Post    0.402858  float64
1  Longitude    0.402233  float64
2   Latitude    0.402233  float64
3        Age    0.000129  float64
                var  proportion    dtype
4              Post    0.402858  float64
5      Neighborhood    0.402749   object
3          District    0.402729   object
1    ArrestLocation    0.402630   object
6         Longitude    0.402233  float64
7          Latitude    0.402233  float64
2  IncidentLocation    0.365831   object
0               Age    0.000129  float64


### first approach remove missing data

In [256]:
# Remove rows with missing values from df_train
df_train = df_train.dropna(subset=np.intersect1d(df_miss['var'], df_train.columns),
                           inplace=False)

# Remove rows with missing values from df_valid
df_valid = df_valid.dropna(subset=np.intersect1d(df_miss['var'], df_valid.columns),
                           inplace=False)

# Remove rows with missing values from df_test
df_test = df_test.dropna(subset=np.intersect1d(df_miss['var'], df_test.columns),
                         inplace=False)

# Combine df_train, df_valid and df_test
df = pd.concat([df_train, df_valid, df_test], sort=False)
print(nan_checker(df))
print(df.shape)
df.head()

                var  proportion   dtype
1  IncidentLocation    0.032228  object
0    ArrestLocation    0.001114  object
2      Neighborhood    0.000033  object
(60165, 22)


,Age,Sex,Race,ArrestLocation,IncidentLocation,Charge,District,Post,Neighborhood,Longitude,Latitude,Season,Daynight,Max.TemperatureF,Mean.TemperatureF,Min.TemperatureF,PrecipitationIn,datetime_year,datetime_month,datetime_day,datetime_hour,datetime_minute
1,22.0,M,B,1900 W NORTH AVE,W North Av & N Fulton Av,43550,Western,732.0,MONDAWMIN,-76.648292,39.309859,Spring,Night,51,40.9,33,0.00,2016,3,21,23,8
4,52.0,M,W,1500 CHARLOTTE AVE,1500 Charlotte Av,43550,Southeastern,234.0,BROENING MANOR,-76.540519,39.276046,Winter,Day,49,36,22,0,2013,12,27,11,0
6,23.0,M,B,400 VENABLE AVE,400 Venable Av,11415,Northern,526.0,Oakenshawe,-76.610964,39.329273,Winter,Night,51,41,31,0,2013,3,20,19,40
7,19.0,M,B,800 STOLL ST,800 Stoll St,11635,Southern,913.0,BROOKLYN,-76.600611,39.230925,Summer,Night,78,66,54,0,2014,8,15,21,30
8,27.0,M,W,5600 ODONNELL ST,5600 O''Donnell St,43550,Southeastern,233.0,MEDFORD,-76.547397,39.278772,Summer,Day,89,80,70,0,2015,7,7,14,15


In [257]:
print(df_train.shape)
print(df_valid.shape)
print(df_test.shape)
print(df.shape)

(43349, 22)
(10789, 22)
(6027, 22)
(60165, 22)


## Encoding the data

In [0]:
# find categorical variables
def cat_var_checker(df):
    """
    The categorical variable checker

    Parameters
    ----------
    df: the dataframe
    
    Returns
    ----------
    The dataframe of categorical variables and their number of unique value
    """
    
    # Get the dataframe of categorical variables and their number of unique value
    df_cat = pd.DataFrame([[var, df[var].nunique(dropna=False)]
                           for var in df.columns if df[var].dtype == 'object'],
                          columns=['var', 'nunique'])
    
    # Sort df_cat in accending order of the number of unique value
    df_cat = df_cat.sort_values(by='nunique', ascending=False)
    
    return df_cat

### Handle categorical variables

In [259]:
# count number of different categories in each categorical variable

print('number of unique category:\
\nArrestLocation : {} \
\nIncidentLocation : {} \
\nDistrict : {} \
\nNeighborhood : {} \
\nPrecipitationIn: {}'.format(df['ArrestLocation'].nunique(),df['IncidentLocation'].nunique(),df['District'].nunique(),df['Neighborhood'].nunique(),df['PrecipitationIn'].nunique()))

number of unique category:
ArrestLocation : 10114 
IncidentLocation : 14878 
District : 9 
Neighborhood : 545 
PrecipitationIn: 215


## Find categorical features

In [260]:
# Call cat_var_checker on df
df_cat = cat_var_checker(df)

# Print the dataframe
df_cat.reset_index(drop=True)

,var,nunique
0,IncidentLocation,14879
1,ArrestLocation,10115
2,Neighborhood,546
3,Mean.TemperatureF,497
4,PrecipitationIn,215
5,Max.TemperatureF,184
6,Min.TemperatureF,178
7,Charge,15
8,District,9
9,Race,5


## Convert temperature and precipitation to numeric values

In [261]:
# convert temperatures and precipitation to float
df['Max.TemperatureF'] = df['Max.TemperatureF'].apply(lambda x: float(str(x).replace('\xa0','')))
df['Mean.TemperatureF'] = df['Mean.TemperatureF'].apply(lambda x: float(str(x).replace('\xa0','')))
df['Min.TemperatureF'] = df['Min.TemperatureF'].apply(lambda x: float(str(x).replace('\xa0','')))
df['PrecipitationIn'] = df['PrecipitationIn'].apply(lambda x: float(str(x).replace('\xa0','')))
df.tail()

,Age,Sex,Race,ArrestLocation,IncidentLocation,Charge,District,Post,Neighborhood,Longitude,Latitude,Season,Daynight,Max.TemperatureF,Mean.TemperatureF,Min.TemperatureF,PrecipitationIn,datetime_year,datetime_month,datetime_day,datetime_hour,datetime_minute
10071,51.0,F,B,3700 FAIRHAVEN AVE,3700 Fairhaven Av,43550,Southern,911.0,Curtis Bay,-76.590517,39.231192,Spring,Night,68.0,58.0,47.0,0.0,2013,5,1,18,0
10073,70.0,M,B,4300 NORFOLK AVE,4300 Norfolk Av,43550,Northwestern,622.0,WEST FOREST PARK,-76.689840,39.322704,Winter,Night,32.0,19.0,6.0,0.0,2014,1,4,20,0
10074,52.0,M,B,UNIT W 20TH ST,1300 Pontiac Av,11415,Northern,516.0,Charles North,-76.617028,39.312180,Spring,Day,48.0,38.0,28.0,0.0,2013,4,2,7,45
10075,29.0,M,B,4000 W BAY AVE,4000 West Bay Av,43550,Southern,911.0,BROOKLYN,-76.594894,39.229789,Fall,Day,80.0,66.0,52.0,0.0,2015,9,24,14,45
10076,34.0,M,B,400 E BALTIMORE ST,400 E Baltimore St,11415,Central,111.0,Downtown,-76.609632,39.289780,Spring,Night,68.0,53.0,38.0,0.0,2013,4,7,2,30


## Check the categorical result after covert

In [262]:
# Call cat_var_checker on df
df_cat = cat_var_checker(df)

# Print the dataframe
df_cat.reset_index(drop=True)

,var,nunique
0,IncidentLocation,14879
1,ArrestLocation,10115
2,Neighborhood,546
3,Charge,15
4,District,9
5,Race,5
6,Season,4
7,Sex,2
8,Daynight,2


## Encode categorical features

In [263]:
# handle feature categories

# One-hot-encode the categorical features in the combined training and testing data
# use get_dummies on categorical variables with small number of unique levels
df = pd.get_dummies(df, columns=np.setdiff1d(df_cat['var'], [target, 'ArrestLocation','IncidentLocation','Neighborhood']))

# handle target and 'ArrestLocation','IncidentLocation','Neighborhood '
# use labelEncoder for large categorical variables
from sklearn.preprocessing import LabelEncoder

# The LabelEncoder
le = LabelEncoder()

# Encode the categorical ArrestLocation in the combined data
df['ArrestLocation'] = le.fit_transform(df['ArrestLocation'].astype(str))

# Encode the categorical IncidentLocation in the combined data
df['IncidentLocation'] = le.fit_transform(df['IncidentLocation'].astype(str))

# Encode the categorical Neighborhood in the combined data
df['Neighborhood'] = le.fit_transform(df['Neighborhood'].astype(str))

# Encode the categorical target in the combined data
df[target] = le.fit_transform(df[target].astype(str))

# Print the first 5 rows of df
df.head()

,Age,ArrestLocation,IncidentLocation,Charge,Post,Neighborhood,Longitude,Latitude,Max.TemperatureF,Mean.TemperatureF,Min.TemperatureF,PrecipitationIn,datetime_year,datetime_month,datetime_day,datetime_hour,datetime_minute,Daynight_Day,Daynight_Night,District_Central,District_Eastern,District_Northeastern,District_Northern,District_Northwestern,District_Southeastern,District_Southern,District_Southwestern,District_Western,Race_A,Race_B,Race_I,Race_U,Race_W,Season_Fall,Season_Spring,Season_Summer,Season_Winter,Sex_F,Sex_M
1,22.0,2755,14715,14,732.0,302,-76.648292,39.309859,51.0,40.9,33.0,0.0,2016,3,21,23,8,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1
4,52.0,1598,1972,14,234.0,29,-76.540519,39.276046,49.0,36.0,22.0,0.0,2013,12,27,11,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,1
6,23.0,6682,8665,6,526.0,361,-76.610964,39.329273,51.0,41.0,31.0,0.0,2013,3,20,19,40,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1
7,19.0,9609,12493,8,913.0,30,-76.600611,39.230925,78.0,66.0,54.0,0.0,2014,8,15,21,30,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1
8,27.0,8368,10896,14,233.0,294,-76.547397,39.278772,89.0,80.0,70.0,0.0,2015,7,7,14,15,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1


## Getting the name of the features

In [264]:

# Separating the training data
df_train = df.iloc[:df_train.shape[0], :].copy(deep=True)

# Separating the validation data
df_valid = df.iloc[df_train.shape[0]:df_train.shape[0] + df_valid.shape[0], :].copy(deep=True)

# Separating the testing data
df_test = df.iloc[df_train.shape[0] + df_valid.shape[0]:, :].copy(deep=True)

df_test.head()
features = np.setdiff1d(df.columns, [target])
print(df_train.shape)
print(df_valid.shape)
print(df_test.shape)
print(df.shape)
print(features)

(43349, 39)
(10789, 39)
(6027, 39)
(60165, 39)
['Age' 'ArrestLocation' 'Daynight_Day' 'Daynight_Night' 'District_Central'
 'District_Eastern' 'District_Northeastern' 'District_Northern'
 'District_Northwestern' 'District_Southeastern' 'District_Southern'
 'District_Southwestern' 'District_Western' 'IncidentLocation' 'Latitude'
 'Longitude' 'Max.TemperatureF' 'Mean.TemperatureF' 'Min.TemperatureF'
 'Neighborhood' 'Post' 'PrecipitationIn' 'Race_A' 'Race_B' 'Race_I'
 'Race_U' 'Race_W' 'Season_Fall' 'Season_Spring' 'Season_Summer'
 'Season_Winter' 'Sex_F' 'Sex_M' 'datetime_day' 'datetime_hour'
 'datetime_minute' 'datetime_month' 'datetime_year']


## check train dataset

In [265]:
print(df_train.shape)
df_train.head()

(43349, 39)


,Age,ArrestLocation,IncidentLocation,Charge,Post,Neighborhood,Longitude,Latitude,Max.TemperatureF,Mean.TemperatureF,Min.TemperatureF,PrecipitationIn,datetime_year,datetime_month,datetime_day,datetime_hour,datetime_minute,Daynight_Day,Daynight_Night,District_Central,District_Eastern,District_Northeastern,District_Northern,District_Northwestern,District_Southeastern,District_Southern,District_Southwestern,District_Western,Race_A,Race_B,Race_I,Race_U,Race_W,Season_Fall,Season_Spring,Season_Summer,Season_Winter,Sex_F,Sex_M
1,22.0,2755,14715,14,732.0,302,-76.648292,39.309859,51.0,40.9,33.0,0.0,2016,3,21,23,8,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1
4,52.0,1598,1972,14,234.0,29,-76.540519,39.276046,49.0,36.0,22.0,0.0,2013,12,27,11,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,1
6,23.0,6682,8665,6,526.0,361,-76.610964,39.329273,51.0,41.0,31.0,0.0,2013,3,20,19,40,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1
7,19.0,9609,12493,8,913.0,30,-76.600611,39.230925,78.0,66.0,54.0,0.0,2014,8,15,21,30,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1
8,27.0,8368,10896,14,233.0,294,-76.547397,39.278772,89.0,80.0,70.0,0.0,2015,7,7,14,15,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1


## check valid dataset

In [266]:
print(df_valid.shape)
df_valid.head()

(10789, 39)


,Age,ArrestLocation,IncidentLocation,Charge,Post,Neighborhood,Longitude,Latitude,Max.TemperatureF,Mean.TemperatureF,Min.TemperatureF,PrecipitationIn,datetime_year,datetime_month,datetime_day,datetime_hour,datetime_minute,Daynight_Day,Daynight_Night,District_Central,District_Eastern,District_Northeastern,District_Northern,District_Northwestern,District_Southeastern,District_Southern,District_Southwestern,District_Western,Race_A,Race_B,Race_I,Race_U,Race_W,Season_Fall,Season_Spring,Season_Summer,Season_Winter,Sex_F,Sex_M
0,27.0,4101,5200,8,334.0,314,-76.580779,39.298934,86.0,74.0,61.0,0.00,2013,6,22,13,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1
1,52.0,3659,4612,14,221.0,316,-76.584947,39.297654,65.0,59.6,54.0,0.00,2017,9,30,22,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1
3,21.0,1429,1751,9,312.0,132,-76.599223,39.311962,90.0,75.0,59.0,0.00,2014,6,16,10,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0
4,26.0,9099,11827,5,424.0,340,-76.551041,39.364399,69.0,64.0,58.0,0.54,2014,10,7,16,30,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0
5,18.0,4139,5260,8,411.0,103,-76.596706,39.320787,74.0,66.0,57.0,0.04,2013,4,18,22,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1


## check test dataset

In [267]:
print(df_test.shape)
df_test.head()

(6027, 39)


,Age,ArrestLocation,IncidentLocation,Charge,Post,Neighborhood,Longitude,Latitude,Max.TemperatureF,Mean.TemperatureF,Min.TemperatureF,PrecipitationIn,datetime_year,datetime_month,datetime_day,datetime_hour,datetime_minute,Daynight_Day,Daynight_Night,District_Central,District_Eastern,District_Northeastern,District_Northern,District_Northwestern,District_Southeastern,District_Southern,District_Southwestern,District_Western,Race_A,Race_B,Race_I,Race_U,Race_W,Season_Fall,Season_Spring,Season_Summer,Season_Winter,Sex_F,Sex_M
0,20.0,174,153,13,912.0,51,-76.611012,39.235427,93.0,84.0,74.0,0.00,2013,7,20,15,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0
1,53.0,5102,6555,14,633.0,407,-76.672573,39.350832,62.0,54.0,46.0,0.00,2013,4,23,9,5,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1
3,26.0,1391,1701,13,832.0,491,-76.666634,39.261553,30.0,22.0,14.0,0.00,2014,2,11,22,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,1
4,26.0,3195,3973,6,811.0,495,-76.665885,39.310311,48.0,46.0,44.0,0.02,2017,1,21,23,15,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,1,0
5,38.0,4781,6138,8,121.0,117,-76.621865,39.292950,89.0,76.0,63.0,0.00,2014,8,5,18,23,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1


## Getting the feature matrix and target vector

In [0]:
# convert to numpy data types
# Get the feature matrix
X_train = df_train[features].to_numpy()
X_valid = df_valid[features].to_numpy()
X_test = df_test[features].to_numpy()

# Get the target vector
y_train = df_train[target].astype(int).to_numpy()
y_valid = df_valid[target].astype(int).to_numpy()


## Scaling the data

In [0]:
# Scale data for better prediction
from sklearn.preprocessing import StandardScaler

# The StandardScaler
ss = StandardScaler()

# Standardize the training data
X_train = ss.fit_transform(X_train)

# Standardize the validation data
X_valid = ss.transform(X_valid)

# Standardize the testing data
X_test = ss.transform(X_test)

# Hyperparameter Tuning and Model Selection

## Creating the dictionary of the models
- In the dictionary:
    - the key is the acronym of the model
    - the value is the model

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.tree import DecisionTreeClassifier

models = {'lr': LogisticRegression(class_weight='balanced', random_state=42),
          'dtc': DecisionTreeClassifier(class_weight='balanced', random_state=42),
          'rfc': RandomForestClassifier(class_weight='balanced', random_state=42),
          'hgbc': HistGradientBoostingClassifier(random_state=42),
          'mlpc': MLPClassifier(early_stopping=True, random_state=42),
          'xgbc': XGBClassifier(seed=42)}
# The MLPClassifier
# models = {'rfc': RandomForestClassifier(class_weight='balanced', random_state=42),
#           'mlpc': MLPClassifier(early_stopping=True, random_state=42)}

## Creating the dictionary of the pipelines
In the dictionary:
- the key is the acronym of the model
- the value is the pipeline, which, for now, only includes the model

In [0]:
# build pipeline for each model
from sklearn.pipeline import Pipeline

pipes = {}

for acronym, model in models.items():
    pipes[acronym] = Pipeline([('model', model)])

## Hyperparameter tuning and  model selection using Sklearn (0.22 or above)

### Getting the predefined split cross-validator

In [283]:

from sklearn.model_selection import PredefinedSplit
import numpy as np

# Combine the feature matrix in the training and validation data
X_train_valid = np.vstack((X_train, X_valid))

# Combine the target vector in the training and validation data
y_train_valid = np.append(y_train, y_valid)

# Get the indices of training and validation data
train_valid_idxs = np.append(np.full(X_train.shape[0], -1), np.full(X_valid.shape[0], 0))

# The PredefinedSplit
ps = PredefinedSplit(train_valid_idxs)
print(ps)

PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0]))


### Creating the dictionary of the parameter grids
- In the dictionary:
    - the key is the acronym of the model
    - the value is the parameter grid of the model

In [0]:
# instantiate parameter grids
param_grids = {}


#### The parameter grid for logistic regression
The hyperparameters we want to fine-tune are:
- C
- tol

See details of the meaning of the hyperparametes in sklearn.linear_model.LogisticRegression

In [0]:

# The grids for C
C_grids = [10 ** i for i in range(-2, 3)]

# The grids for tol
tol_grids = [10 ** i for i in range(-6, -1)]

# Update param_grids
param_grids['lr'] = [{'model__C': C_grids,
                      'model__tol': tol_grids}]

#### The parameter grid for decision tree
The hyperparameters we want to fine-tune are:
- min_samples_split
- min_samples_leaf

See details of the meaning of the hyperparametes in sklearn.tree.DecisionTreeClassifier

In [0]:
# The grids for min_samples_split
min_samples_split_grids = [2, 20, 100]

# The grids for min_samples_leaf
min_samples_leaf_grids = [1, 20, 100]

# Update param_grids
param_grids['dtc'] = [{'model__min_samples_split': min_samples_split_grids,
                       'model__min_samples_leaf': min_samples_leaf_grids}]

#### The parameter grid for Adaboost
The hyperparameters we want to fine-tune are:
- learning_rate

See details of the meaning of the hyperparametes in sklearn.ensemble.AdaBoostClassifier

In [0]:
# The grids for learning_rate
learning_rate_grids = [10 ** i for i in range(-3, 2)]

# Update param_grids
param_grids['abc'] = [{'model__learning_rate': learning_rate_grids}]

#### The parameter grid for HistGradientBoosting
The hyperparameters we want to fine-tune are:
- learning_rate
- min_samples_leaf

See details of the meaning of the hyperparametes in sklearn.ensemble.HistGradientBoostingClassifier

In [0]:
# The grids for learning_rate
learning_rate_grids = [10 ** i for i in range(-3, 2)]

# The grids for min_samples_leaf
min_samples_leaf_grids = [1, 20, 100]

# Update param_grids
param_grids['hgbc'] = [{'model__learning_rate': learning_rate_grids,
                        'model__min_samples_leaf': min_samples_leaf_grids}]

#### The parameter grid for XGBoost
The hyperparameters we want to fine-tune are:
- eta
- gamma
- lambda

See details of the meaning of the hyperparametes in XGBoost Parameters

In [0]:
# The grids for eta
eta_grids = [10 ** i for i in range(-4, 1)]

# The grids for gamma
gamma_grids = [0, 10, 100]

# The grids for lambda
lambda_grids = [10 ** i for i in range(-4, 5)]

# Update param_grids
param_grids['xgbc'] = [{'model__eta': eta_grids,
                        'model__gamma': gamma_grids,
                        'model__lambda': lambda_grids}]

#### The parameter grid for random forest
The hyperparameters we want to fine-tune are:
- min_samples_split
- min_samples_leaf

See details of the meaning of the hyperparametes in [sklearn.ensemble.RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

In [0]:
# The grids for min_samples_split
min_samples_split_grids = [2, 20, 100]

# The grids for min_samples_leaf
min_samples_leaf_grids = [1, 20, 100]

# Update param_grids
param_grids['rfc'] = [{'model__min_samples_split': min_samples_split_grids,
                       'model__min_samples_leaf': min_samples_leaf_grids}]

#### The parameter grid for multi-layer perceptron classifier
The hyperparameters we want to fine-tune are:
- alpha
- learning_rate_init

See details of the meaning of the hyperparametes in [sklearn.neural_network.MLPClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)

In [291]:

# # # The grids for alpha
# alpha_grids = [10 ** i for i in range(-6, -1)]

# # The grids for learning_rate_init
# learning_rate_init_grids = [10 ** i for i in range(-5, 0)]

# extra grids for better score

# The grids for hidden_layer_sizes_grids
# hidden_layer_sizes_grids =  [(50,50,50), (50,100,50), (100,)]

# The grids for activation_grids
# activation_grids = ['tanh', 'relu']
# activation_grids = ['relu']

# The grids for solver_grids
solver_grids = ['sgd', 'adam']

# Update param_grids
# param_grids['mlpc'] = [{
#     'model__alpha':alpha_grids,
#     'model__learning_rate':learning_rate_init_grids,
#     'model__hidden_layer_sizes':hidden_layer_sizes_grids,
#     'model__activation':activation_grids,
#     'model__solver':solver_grids
#     }]
# print(param_grids['mlpc'])

# The grids for alpha
alpha_grids = [10 ** i for i in range(-6, -1)]

# The grids for learning_rate_init
learning_rate_init_grids = [10 ** i for i in range(-5, 0)]

# Update param_grids
param_grids['mlpc'] = [{'model__alpha': alpha_grids,
                        'model__learning_rate_init': learning_rate_init_grids}]
print(param_grids)

{'lr': [{'model__C': [0.01, 0.1, 1, 10, 100], 'model__tol': [1e-06, 1e-05, 0.0001, 0.001, 0.01]}], 'dtc': [{'model__min_samples_split': [2, 20, 100], 'model__min_samples_leaf': [1, 20, 100]}], 'abc': [{'model__learning_rate': [0.001, 0.01, 0.1, 1, 10]}], 'hgbc': [{'model__learning_rate': [0.001, 0.01, 0.1, 1, 10], 'model__min_samples_leaf': [1, 20, 100]}], 'xgbc': [{'model__eta': [0.0001, 0.001, 0.01, 0.1, 1], 'model__gamma': [0, 10, 100], 'model__lambda': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]}], 'rfc': [{'model__min_samples_split': [2, 20, 100], 'model__min_samples_leaf': [1, 20, 100]}], 'mlpc': [{'model__alpha': [1e-06, 1e-05, 0.0001, 0.001, 0.01], 'model__learning_rate_init': [1e-05, 0.0001, 0.001, 0.01, 0.1]}]}


### Creating the directory for the cv results.

In [0]:
import os

# Make directory
directory = os.path.dirname('./cv_results/')
if not os.path.exists(directory):
    os.makedirs(directory)

### Hyperparameter Tuning

In [0]:

from sklearn.model_selection import GridSearchCV

# The list of [best_score_, best_params_, best_estimator_] obtained by GridSearchCV
best_score_param_estimator_gs = []

for acronym in pipes.keys():
    # GridSearchCV
    gs = GridSearchCV(estimator=pipes[acronym],
                      param_grid=param_grids[acronym],
                      scoring='f1_micro',
                      n_jobs=-1,
                      cv=ps,
                      return_train_score=True)
        
    # Fit the pipeline
    gs = gs.fit(X_train_valid, y_train_valid)
    
    # Update best_score_param_estimator_gs
    best_score_param_estimator_gs.append([gs.best_score_, gs.best_params_, gs.best_estimator_])
    
    # Sort cv_results in ascending order of 'rank_test_score' and 'std_test_score'
    cv_results = pd.DataFrame.from_dict(gs.cv_results_).sort_values(by=['rank_test_score', 'std_test_score'])
    
    # Get the important columns in cv_results
    important_columns = ['rank_test_score',
                         'mean_test_score', 
                         'std_test_score', 
                         'mean_train_score', 
                         'std_train_score',
                         'mean_fit_time', 
                         'std_fit_time',                        
                         'mean_score_time', 
                         'std_score_time']
    
    # Move the important columns ahead
    cv_results = cv_results[important_columns + sorted(list(set(cv_results.columns) - set(important_columns)))]

    # Write cv_results file
    cv_results.to_csv(path_or_buf='./cv_results/' + acronym + '.csv', index=False)

### Model Selection

In [293]:
# Sort best_score_param_estimator_gs in descending order of the best_score_
best_score_param_estimator_gs = sorted(best_score_param_estimator_gs, key=lambda x : x[0], reverse=True)

# Print best_score_param_estimator_gs
pd.DataFrame(best_score_param_estimator_gs, columns=['best_score', 'best_param', 'best_estimator'])

,best_score,best_param,best_estimator
0,0.477431,"{'model__min_samples_leaf': 1, 'model__min_sam...","((DecisionTreeClassifier(ccp_alpha=0.0, class_..."
1,0.437946,"{'model__eta': 0.0001, 'model__gamma': 0, 'mod...","(XGBClassifier(base_score=0.5, booster='gbtree..."
2,0.433497,"{'model__learning_rate': 0.01, 'model__min_sam...",(HistGradientBoostingClassifier(l2_regularizat...
3,0.430160,"{'model__alpha': 0.01, 'model__learning_rate_i...","(MLPClassifier(activation='relu', alpha=0.01, ..."
4,0.324497,"{'model__min_samples_leaf': 1, 'model__min_sam...","(DecisionTreeClassifier(ccp_alpha=0.0, class_w..."
5,0.179998,"{'model__C': 0.01, 'model__tol': 1e-06}","(LogisticRegression(C=0.01, class_weight='bala..."


# Testing the best fitted model

In [294]:
# Get the best_score, best_param and best_estimator obtained by GridSearchCV
best_score_gs, best_param_gs, best_estimator_gs = best_score_param_estimator_gs[0]

# Get the prediction on the testing data using best_model
y_test_pred = best_estimator_gs.predict(X_test)

# Transform y_test_pred back to the original class
y_test_pred = le.inverse_transform(y_test_pred)
# print(y_test_pred)

y_test = df_test[target].to_numpy()
# y_test = le.inverse_transform(y_test)
print(X_test)
print(y_test)
df_test.tail()

[[-1.12294737 -1.63693271  0.97199457 ... -1.29845015  0.27453906
  -1.04723983]
 [ 1.63330337  0.07018922  0.97199457 ... -1.01549101 -0.62851086
  -1.04723983]
 [-0.62181087 -1.21534842 -1.02881233 ... -1.29845015 -1.23054414
  -0.31673006]
 ...
 [ 1.54978062  1.7949782   0.97199457 ...  1.24818213 -0.62851086
  -1.04723983]
 [-0.37124262  0.67190813  0.97199457 ...  1.24818213  0.87657234
   0.41377972]
 [ 0.04637113  0.5378464  -1.02881233 ...  0.3993047  -0.62851086
  -1.04723983]]
[13 14 13 ...  6 14  6]


,Age,ArrestLocation,IncidentLocation,Charge,Post,Neighborhood,Longitude,Latitude,Max.TemperatureF,Mean.TemperatureF,Min.TemperatureF,PrecipitationIn,datetime_year,datetime_month,datetime_day,datetime_hour,datetime_minute,Daynight_Day,Daynight_Night,District_Central,District_Eastern,District_Northeastern,District_Northern,District_Northwestern,District_Southeastern,District_Southern,District_Southwestern,District_Western,Race_A,Race_B,Race_I,Race_U,Race_W,Season_Fall,Season_Spring,Season_Summer,Season_Winter,Sex_F,Sex_M
10071,51.0,6032,7829,14,911.0,110,-76.590517,39.231192,68.0,58.0,47.0,0.0,2013,5,1,18,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0
10073,70.0,7166,9266,14,622.0,501,-76.689840,39.322704,32.0,19.0,6.0,0.0,2014,1,4,20,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1
10074,52.0,10081,1614,6,516.0,96,-76.617028,39.312180,48.0,38.0,28.0,0.0,2013,4,2,7,45,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1
10075,29.0,6839,8872,14,911.0,30,-76.594894,39.229789,80.0,66.0,52.0,0.0,2015,9,24,14,45,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1
10076,34.0,6452,8375,6,111.0,126,-76.609632,39.289780,68.0,53.0,38.0,0.0,2013,4,7,2,30,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1
